In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv('/kaggle/input/nasaexoplanets/cleaned_5250.csv')
df.reset_index(inplace=True, drop=True)

df.head()

In [ ]:
df.describe()

In [ ]:
df.isna().sum() #stellar_magnitude,distance,mass_multiplier,

In [ ]:
df_without_miss = df.dropna()

df_without_miss.describe()

In [ ]:
import matplotlib.pyplot as plt

plt.boxplot(df_without_miss['discovery_year'])

In [ ]:
import seaborn as sns
sns.scatterplot(x = "distance", y = "discovery_year", hue = "planet_type", style = "planet_type",data = df);

In [ ]:
import seaborn as sns
sns.scatterplot(x = "distance", y = "discovery_year", hue = "planet_type", style = "planet_type",data = df_without_miss);

In [ ]:
pip install fancyimpute


In [ ]:
from fancyimpute import KNN
# Initialize KNN
knn_imputer = KNN()
new_df2 = df.copy()
new_df2 = new_df2.drop(new_df2[['planet_type','name','mass_wrt','radius_wrt','detection_method']],axis=1)

# Impute using fit_tranform 
new_df2.iloc[:, :] = knn_imputer.fit_transform(new_df2)

new_df2.head()

In [ ]:
x = new_df2
dms = df[['planet_type','name','mass_wrt','radius_wrt','detection_method']]
X_ = pd.get_dummies(dms)
X = pd.concat([x,X_],axis=1)
Y = df['planet_type']

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X,Y,test_size=0.3,random_state=38) 

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(x_train)
X_test = sc.transform(x_test)

In [ ]:
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score
lgbm = LGBMClassifier().fit(X_train,y_train)
y_pred = lgbm.predict(X_test)
accuracy_score(y_test,y_pred)